In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # Alignments 

import seaborn as sns 
import pandas as pd
import requests

In [2]:
new_data = pd.read_csv('/Users/riyamhatre/Desktop/scraped_data.csv').drop(columns = {'Unnamed: 0'})

In [5]:
df = new_data

import requests

def get_imdb_rating(movie_name, api_key):
    url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
    response = requests.get(url)
    
    data = response.json()

    # Check if movie is found
    if data.get("Response") == "True":
        return data.get("imdbRating", "Rating not available")
    else:
        return ""

def get_box_office(movie_name, api_key):
    url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    if data.get("Response") == "True":
        if 'BoxOffice' in data.keys():
            return data['BoxOffice']
        if 'Awards' in data.keys():
            return data['Awards']
    else:
        return ''
    

def get_awards(movie_name, api_key):
    url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    if data.get("Response") == "True":
        if 'Awards' in data.keys():
            return data['Awards']
    else:
        return ''
    
def get_others(movie_name, api_key):
    url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    if data.get("Response") == "True":
            return [data['Genre'], data['Language'], data['imdbVotes']]
    else:
        return f"Movie '{movie_name}' not found."

lst = df['title'].to_list()
ratings = []
box_office = []
genre = []
language = []
imdbVotes = []
awards = []
for i in lst:
    ratings.append(get_imdb_rating(i,'76a00330'))
    box_office.append(get_box_office(i, '76a00330'))
    awards.append(get_awards(i, '76a00330'))
    genre.append(get_others(i, '76a00330')[0])
    language.append(get_others(i, '76a00330')[1])
    imdbVotes.append(get_others(i, '76a00330')[2])

df['imdb_ratings'] = ratings
df['genre'] = genre
df['language'] = language
df['imdb_votes'] = imdbVotes
df['box_office'] = box_office
df['awards'] = awards

In [52]:
new_data.set_index('show_id', inplace=True)
df.set_index('show_id', inplace=True)
new_data.update(df)
new_data.reset_index(inplace=True)

In [53]:
#Export Data
from pathlib import Path
filepath = Path('/Users/riyamhatre/Desktop/scraped_data.csv')
filepath.parent.mkdir(parents=True, exist_ok = True)
new_data.to_csv(filepath)